In [ ]:
#import libs
import pandas as pd
import sqlite3

from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.python import PythonOperator

#connect to DB
CON = sqlite3.connect('example.db')
DATE = '2021-01-01'


# extract currency from site
def extract_rate(date, rate_file, **context) -> None:
  """ Extract currency rate from exchange"""
  base = 'EUR'
  symbols = 'USD'
  format = 'csv'
  url = f'https://api.exchangerate.host/timeseries?start_date={date}&end_date={date}&base={base}&symbols={symbols}&format={format}'

  pd.read_csv(url).to_csv(rate_file)


# extract data from GIT
def extract_data(date, data_file, **context) -> None:
  """date in string format yyyy-mm-dd"""
  url = f'https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/{date}.csv'
  
  data_git = pd.read_csv(url).to_csv(data_file)


# load to db
def load_data(currency_dict, conn, **context) -> None:
    """ Load to DB """
    for k,v in currency_dict.items():
      data = pd.read_csv(v)
      data.to_sql(k, conn, if_exists='replace', index=False)


# DAG
dag = DAG(dag_id='dag',
          default_args={'owner':'airflow'},
          schedule_interval='@daily',
          start_date=days_ago(1))

extract_rate = PythonOperator(
    task_id='extract_rate',
    python_callable=extract_rate,
    dag=dag,
    op_kwargs={
      'date': DATE,
      'rate_file': '/tmp/rate.csv'
    }
)

extract_data = PythonOperator(
    task_id='extract_data',
    python_callable=extract_data,
    dag=dag,
    op_kwargs={
      'date': DATE,
      'data_file': '/tmp/data.csv'
    }
)

load_data = PythonOperator(
    task_id='load_data',
    python_callable=load_data,
    dag=dag,
    op_kwargs={
      'currency_dict': {
          'rate': '/tmp/rate.csv',
          'data': '/tmp/data.csv'
      },
      'conn': CON
    }
)

# sequence
[extract_rate, extract_data] >> load_data